In [1]:
N, D_in, H, D_out = 64, 3 , 5 , 2

In [4]:
!pip install torch

     |████████████████████████████████| 881.9 MB 244 bytes/s  0:00:011     |█████████████▊                  | 378.5 MB 10.3 MB/s eta 0:00:49     |██████████████████████▋         | 624.1 MB 6.7 MB/s eta 0:00:39


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
import random
import numpy as np
from tqdm.auto import tqdm

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [19]:
def load_data():
    data = pd.read_csv('../../data/dataset_annotated_v1.csv')
    data = shuffle(data)
    X1 = data[["feature_1", "feature_2", "feature_3"]].to_numpy().astype(np.float32)
    y1 = data["presence"].to_numpy()
    
    data = pd.read_csv('../../data/dataset_annotated_v2.csv')
    data = shuffle(data)
    X2 = data[["feature_1", "feature_2", "feature_3"]].to_numpy().astype(np.float32)
    y2 = data["presence"].to_numpy()

    data = pd.read_csv('../../data/dataset_annotated_v3.csv')
    data = shuffle(data)
    X3 = data[["feature_1", "feature_2", "feature_3"]].to_numpy().astype(np.float32)
    y3 = data["presence"].to_numpy()
    
    X = np.concatenate((X1, X2, X3), axis=0)
    y = np.concatenate((y1, y2, y3), axis=None)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    return X_train, X_test, y_train, y_test

In [20]:
X_train, X_test, y_train, y_test = load_data()

In [21]:
class ANNDataSet(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]), torch.tensor(self.y[idx])
    def __len__(self):
        return len(self.x)

In [22]:
class ANN(torch.nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.l1 = torch.nn.Linear(D_in, H)
        self.sigmoid = torch.nn.Sigmoid()
        self.l2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        x = self.l1(x)
        x = self.sigmoid(x)
        x = self.l2(x)
        return x

In [23]:
train_data = ANNDataSet(X_train,y_train)
test_data = ANNDataSet(X_test,y_test)
model = ANN()
optimizer = torch.optim.Adam(model.parameters(), 1e-5)
test_data_loader = DataLoader(test_data, batch_size=N)
train_data_loader = DataLoader(train_data, batch_size=N)

In [24]:
model.train()
preds = []
weight = torch.tensor([list(y_train).count(1)/list(y_train).count(0), 1])
for epoch in range(10000):
    preds = []
    total_loss1 = 0
    print(f"Epoch:{epoch}")
    #for batch in tqdm(data_loader):
    for batch in train_data_loader:
    
        x , y  = batch
        logits = model(x)
        loss = torch.nn.functional.cross_entropy(logits, y,weight=weight)
        #torch.argmax(logits, dim=1).flatten().cpu().numpy()
        loss.backward()
        optimizer.step()
        model.zero_grad()
        total_loss1 += loss.detach().item()
    total_loss2 = 0
    #for batch in tqdm(data_loader):
    for batch in test_data_loader:
    
        x , y  = batch
        with torch.no_grad():
          logits = model(x)
          #print(y)
          loss = torch.nn.functional.cross_entropy(logits, y)
        preds.extend(torch.argmax(logits, dim=1).flatten().cpu().numpy().tolist())
        total_loss2 += loss.detach().item()
    print(total_loss1,total_loss2)

Epoch:0
9.247921824455261 5.507834255695343
Epoch:1
9.2461479306221 5.505094230175018
Epoch:2
9.24443644285202 5.502398431301117
Epoch:3
9.242742955684662 5.499729335308075
Epoch:4
9.241060554981232 5.497078359127045
Epoch:5
9.239385902881622 5.494441032409668
Epoch:6
9.237718343734741 5.491814136505127
Epoch:7
9.236055672168732 5.48919552564621
Epoch:8
9.234396874904633 5.486584961414337
Epoch:9
9.232742428779602 5.483981072902679
Epoch:10
9.231091797351837 5.481383442878723
Epoch:11
9.229444801807404 5.478791296482086
Epoch:12
9.227800846099854 5.476204693317413
Epoch:13
9.226159930229187 5.473623096942902
Epoch:14
9.224522411823273 5.471046686172485
Epoch:15
9.222888052463531 5.468474864959717
Epoch:16
9.221256077289581 5.465907514095306
Epoch:17
9.21962708234787 5.463344752788544
Epoch:18
9.218000948429108 5.460786521434784
Epoch:19
9.21637749671936 5.458232283592224
Epoch:20
9.214756727218628 5.455682158470154
Epoch:21
9.213138580322266 5.453136324882507
Epoch:22
9.211523115634918

8.968575716018677 5.067199766635895
Epoch:192
8.967290818691254 5.065169394016266
Epoch:193
8.966007351875305 5.063141465187073
Epoch:194
8.964725732803345 5.061116695404053
Epoch:195
8.963445723056793 5.059094250202179
Epoch:196
8.962167024612427 5.057073950767517
Epoch:197
8.96088981628418 5.05505645275116
Epoch:198
8.959614157676697 5.053041100502014
Epoch:199
8.958339989185333 5.051028490066528
Epoch:200
8.957067728042603 5.049018681049347
Epoch:201
8.955796599388123 5.047011077404022
Epoch:202
8.954527616500854 5.045006036758423
Epoch:203
8.953259587287903 5.043003439903259
Epoch:204
8.95199304819107 5.041003227233887
Epoch:205
8.950728237628937 5.03900545835495
Epoch:206
8.949464917182922 5.037010312080383
Epoch:207
8.94820350408554 5.035017907619476
Epoch:208
8.946943461894989 5.033027768135071
Epoch:209
8.945684373378754 5.031040072441101
Epoch:210
8.944427132606506 5.029054820537567
Epoch:211
8.943171203136444 5.027071952819824
Epoch:212
8.941917419433594 5.025092005729675
Epo

8.759792804718018 4.740082561969757
Epoch:372
8.758744955062866 4.738468766212463
Epoch:373
8.757698118686676 4.736857175827026
Epoch:374
8.756652414798737 4.735248029232025
Epoch:375
8.755607783794403 4.733640432357788
Epoch:376
8.754564464092255 4.732036113739014
Epoch:377
8.753522038459778 4.730433106422424
Epoch:378
8.752480328083038 4.728832125663757
Epoch:379
8.751440107822418 4.7272332310676575
Epoch:380
8.750401020050049 4.725637376308441
Epoch:381
8.74936294555664 4.724042773246765
Epoch:382
8.748326063156128 4.7224509716033936
Epoch:383
8.747289657592773 4.7208608984947205
Epoch:384
8.746255099773407 4.719272673130035
Epoch:385
8.745221018791199 4.717686772346497
Epoch:386
8.74418818950653 4.716103494167328
Epoch:387
8.74315631389618 4.714521646499634
Epoch:388
8.742125809192657 4.712942481040955
Epoch:389
8.741095840930939 4.711364567279816
Epoch:390
8.740067601203918 4.709789514541626
Epoch:391
8.73904001712799 4.708216309547424
Epoch:392
8.738013505935669 4.706645309925079

8.584831833839417 4.479993999004364
Epoch:554
8.583946526050568 4.478741645812988
Epoch:555
8.583061754703522 4.477490961551666
Epoch:556
8.582177937030792 4.476241827011108
Epoch:557
8.581294775009155 4.474994659423828
Epoch:558
8.580412328243256 4.473749399185181
Epoch:559
8.579530239105225 4.472505867481232
Epoch:560
8.5786492228508 4.471264183521271
Epoch:561
8.577768862247467 4.470024287700653
Epoch:562
8.576889216899872 4.468786299228668
Epoch:563
8.57601010799408 4.467550039291382
Epoch:564
8.575131952762604 4.466315448284149
Epoch:565
8.574253976345062 4.465082764625549
Epoch:566
8.573377132415771 4.463851749897003
Epoch:567
8.572500884532928 4.4626225233078
Epoch:568
8.571625232696533 4.4613953828811646
Epoch:569
8.570750415325165 4.460169792175293
Epoch:570
8.569876253604889 4.458946228027344
Epoch:571
8.56900280714035 4.457724094390869
Epoch:572
8.56812983751297 4.4565041065216064
Epoch:573
8.567257463932037 4.455285668373108
Epoch:574
8.566386103630066 4.454068720340729
Epo

8.4290452003479 4.274135112762451
Epoch:742
8.428264677524567 4.273187100887299
Epoch:743
8.427484691143036 4.272240400314331
Epoch:744
8.426705241203308 4.271294891834259
Epoch:745
8.425925850868225 4.270351052284241
Epoch:746
8.42514717578888 4.269408822059631
Epoch:747
8.424369037151337 4.268467724323273
Epoch:748
8.423591256141663 4.267527878284454
Epoch:749
8.422813713550568 4.266589522361755
Epoch:750
8.422036528587341 4.2656524777412415
Epoch:751
8.421260237693787 4.264717221260071
Epoch:752
8.420483410358429 4.263783276081085
Epoch:753
8.419707834720612 4.262850344181061
Epoch:754
8.418932497501373 4.261919319629669
Epoch:755
8.41815733909607 4.260989665985107
Epoch:756
8.417382597923279 4.260061025619507
Epoch:757
8.41660851240158 4.2591336369514465
Epoch:758
8.415834546089172 4.258208096027374
Epoch:759
8.415061354637146 4.257283508777618
Epoch:760
8.414288461208344 4.256360709667206
Epoch:761
8.413515627384186 4.255439281463623
Epoch:762
8.412743151187897 4.254519104957581
E

...

4.561057940125465 2.2186622321605682
Epoch:9143
4.560939192771912 2.2185996025800705
Epoch:9144
4.560820654034615 2.218537151813507
Epoch:9145
4.560702085494995 2.2184745222330093
Epoch:9146
4.560583457350731 2.2184120565652847
Epoch:9147
4.560465082526207 2.2183495610952377
Epoch:9148
4.560346618294716 2.218287095427513
Epoch:9149
4.560228049755096 2.2182246893644333
Epoch:9150
4.560109242796898 2.2181622684001923
Epoch:9151
4.559990882873535 2.2180997133255005
Epoch:9152
4.55987249314785 2.218037247657776
Epoch:9153
4.559754118323326 2.2179747819900513
Epoch:9154
4.559635818004608 2.217912420630455
Epoch:9155
4.559517398476601 2.2178502529859543
Epoch:9156
4.559399142861366 2.2177879959344864
Epoch:9157
4.559280902147293 2.217725694179535
Epoch:9158
4.559162884950638 2.2176635712385178
Epoch:9159
4.559044614434242 2.2176012992858887
Epoch:9160
4.558926284313202 2.2175390273332596
Epoch:9161
4.558808118104935 2.2174768894910812
Epoch:9162
4.558690041303635 2.217414900660515
Epoch:9163

4.540214478969574 2.2077057361602783
Epoch:9323
4.540101423859596 2.207646429538727
Epoch:9324
4.539988562464714 2.2075871974229813
Epoch:9325
4.539875611662865 2.20752777159214
Epoch:9326
4.5397627502679825 2.207468554377556
Epoch:9327
4.539649859070778 2.2074092477560043
Epoch:9328
4.539537101984024 2.2073499262332916
Epoch:9329
4.539424300193787 2.207290753722191
Epoch:9330
4.539311468601227 2.20723132789135
Epoch:9331
4.539198949933052 2.20717217028141
Epoch:9332
4.5390859097242355 2.2071129381656647
Epoch:9333
4.5389735251665115 2.2070538848638535
Epoch:9334
4.538860812783241 2.2069946974515915
Epoch:9335
4.538748115301132 2.2069357484579086
Epoch:9336
4.538635551929474 2.2068765610456467
Epoch:9337
4.53852304816246 2.2068175226449966
Epoch:9338
4.53841070830822 2.206758424639702
Epoch:9339
4.53829799592495 2.2066991180181503
Epoch:9340
4.538185507059097 2.2066400200128555
Epoch:9341
4.538073107600212 2.2065805941820145
Epoch:9342
4.537960782647133 2.2065214216709137
Epoch:9343
4.

4.520362049341202 2.197267383337021
Epoch:9503
4.520254284143448 2.1972108334302902
Epoch:9504
4.520146518945694 2.19715416431427
Epoch:9505
4.520038858056068 2.197097510099411
Epoch:9506
4.5199315547943115 2.1970407515764236
Epoch:9507
4.519823849201202 2.1969839930534363
Epoch:9508
4.519716262817383 2.1969272047281265
Epoch:9509
4.51960888504982 2.1968709379434586
Epoch:9510
4.519501447677612 2.1968142837285995
Epoch:9511
4.519393861293793 2.196757599711418
Epoch:9512
4.519286528229713 2.196701154112816
Epoch:9513
4.519179150462151 2.1966445595026016
Epoch:9514
4.519071772694588 2.1965879052877426
Epoch:9515
4.518964320421219 2.1965312510728836
Epoch:9516
4.518857106566429 2.1964745074510574
Epoch:9517
4.518749907612801 2.1964180171489716
Epoch:9518
4.5186425894498825 2.196361243724823
Epoch:9519
4.518535405397415 2.196304678916931
Epoch:9520
4.518428131937981 2.196248009800911
Epoch:9521
4.518320843577385 2.1961913406848907
Epoch:9522
4.518213719129562 2.196135029196739
Epoch:9523
4

4.501934424042702 2.187564507126808
Epoch:9678
4.501831471920013 2.1875104308128357
Epoch:9679
4.5017285495996475 2.1874564737081528
Epoch:9680
4.50162561237812 2.1874023973941803
Epoch:9681
4.501522794365883 2.187348559498787
Epoch:9682
4.501419946551323 2.1872947365045547
Epoch:9683
4.501317083835602 2.1872407346963882
Epoch:9684
4.501214578747749 2.1871868073940277
Epoch:9685
4.501111716032028 2.187132939696312
Epoch:9686
4.501009002327919 2.187079057097435
Epoch:9687
4.500906318426132 2.187025099992752
Epoch:9688
4.500803560018539 2.1869712322950363
Epoch:9689
4.500701040029526 2.1869173794984818
Epoch:9690
4.500598639249802 2.18686343729496
Epoch:9691
4.500495910644531 2.186809465289116
Epoch:9692
4.500393316149712 2.186755508184433
Epoch:9693
4.500290811061859 2.186701610684395
Epoch:9694
4.500188589096069 2.1866476982831955
Epoch:9695
4.500085815787315 2.186593681573868
Epoch:9696
4.499983489513397 2.1865397542715073
Epoch:9697
4.499880880117416 2.1864857524633408
Epoch:9698
4.4

4.48451167345047 2.178416892886162
Epoch:9851
4.484413057565689 2.178365185856819
Epoch:9852
4.484314322471619 2.1783134788274765
Epoch:9853
4.484215959906578 2.178261935710907
Epoch:9854
4.484117329120636 2.178210198879242
Epoch:9855
4.484018877148628 2.178158715367317
Epoch:9856
4.4839199632406235 2.178107276558876
Epoch:9857
4.483821675181389 2.1780556738376617
Epoch:9858
4.483723312616348 2.1780039221048355
Epoch:9859
4.483624890446663 2.177952542901039
Epoch:9860
4.483526557683945 2.177901029586792
Epoch:9861
4.483428046107292 2.1778495609760284
Epoch:9862
4.483329802751541 2.1777981370687485
Epoch:9863
4.483231529593468 2.1777467876672745
Epoch:9864
4.4831332713365555 2.177695333957672
Epoch:9865
4.48303510248661 2.1776437014341354
Epoch:9866
4.482936903834343 2.1775922924280167
Epoch:9867
4.4828387796878815 2.177540570497513
Epoch:9868
4.482740357518196 2.1774893552064896
Epoch:9869
4.482642278075218 2.17743781208992
Epoch:9870
4.482544168829918 2.1773864179849625
Epoch:9871
4.4

In [25]:
for name , param in model.named_parameters():
  print(name) 
  print(param.detach().numpy().T)

l1.weight
[[-0.24099377  0.09341905  0.27165955  0.83639246  0.6827216 ]
 [ 0.1511586  -0.58199024  0.2952358  -0.13097027 -0.05659134]
 [-1.0417851   0.46101013 -0.70550346  0.6593508   0.72046417]]
l1.bias
[ 0.91676605 -0.846087    0.80878204 -1.0349809  -0.66025865]
l2.weight
[[ 1.3526715  -1.2391232 ]
 [-0.6241882   0.3192553 ]
 [ 0.5866213  -1.0192766 ]
 [-0.54183006  0.47728086]
 [-0.79059637  0.15532656]]
l2.bias
[ 0.38565454 -0.17592695]


In [26]:
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.57      0.77      0.66        53
           1       0.96      0.91      0.93       334

    accuracy                           0.89       387
   macro avg       0.77      0.84      0.79       387
weighted avg       0.91      0.89      0.90       387



In [27]:
torch.save(model.state_dict(), "./ann.pt")